<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT22/blob/PeterTKovacs_lab3/Lab3/PeterTKovacs_lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 3: iterative methods**
**Péter Kovács**

# **Abstract**

In this lab assignment, we venture into the broad field of iterative methods. It is clearly an understatment to write that they are quite common, since they are ubiquitous in practical computing, even for tasks that could be solved in an 'exact' way (systems of linear equations for instance).



#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [58]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2022 Péter Kovács (ptkovacs@kth.se)

# This file is part of the course DD2363 Methods in Scientific Computing
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

In [59]:
# Load neccessary modules.
from google.colab import files
import numpy as np
import theano
import theano.tensor as tt


# **Introduction**

Just as the abstract says, we will be engaged with iterative methods in this chapter. From the simplest case of linear equations, we will proceed to the more complicated nonlinear case.

A generic iterative method reads as:

$x_{k+1}=g(x_k)$ where $g$ is the update function. 

A common example is when we aim at solving the equation $f(x)=0$. Then the choice of $g(x)=x+ \alpha f(x)$ implies that any fixpoint of the iteration indeed solves the original equation.

The convergence of the method is guaranteed - under appropriate technical conditions - by the Banach fixpoint theorem.




# **Method**

### 1) Jacobi iteration

The Jacobi iteration is used to solve systems of linear equations.

Using the intrduction's notation, we would have $f(x)=\mathbf{A}x-b$ so that the solution of the general problem is equivalent to finding the solution of the linear system of equations: $\mathbf{A}x=b$.

To improve convergence speed, a common trick is to 'precondition' the above system ie. to rather approximate the solution of $\mathbf{AB}x=\mathbf{B}b$ where $\mathbf{B}$ is an arbitrary regular matrix. In the Jacobi iteration, it is chosen to be the inverse of the diagonal of $\mathbf{A}$.

Thus the update function: $g(x)=(\mathbf{I}-\mathbf{BA})x+\mathbf{B}b$.

To have a guarantee of convergence, we need to make sure that the update matrix has a norm less than 1.

After ensuring that A is indeed regular (or rather assuming it), we would only need to check the highest singular value of the iteration matrix. But we won't check this, since the main point of iterative methods is that we use them when inverting the matrix would be too costly. So determining the largest singular value is deemed infeasible in this context as well.

At the practical end of matters, we check the residual ie. how much the approximate solution changes in one step. If its norm is below a certain threshold, the iteration is terminated and we have an approximate solution. Also, the case of non-convergent iteration is intended to be detected by investigating the residual norm.

In [60]:

def jacobi(A,b,tol,maxiter=10000):
  """
  perform Jacobi iteration on the system defined by A and b. Iterate until the L2 norm of the residual is smaller than >>tol<< but max >>maxiter<< times

  since we do not check wheter the method is convergent, I had to include manual criteria to prevent overflow: if the residual grows too much,
  the iteration is terminated, assuming it ain't convergent
  """

  B=np.zeros(A.shape,float)
  
  for i in range(A.shape[0]):
    try:
      B[i,i]=1./A[i,i]
    except:
      print('zero in diagonal, cannot apply Jacobi')
  c=B.dot(b)
  M=np.eye(*A.shape,0,float)-B.dot(A)

  j=0
  x=np.zeros(A.shape[0],float)

  while(j<maxiter):

    x_=M.dot(x)+c
    r=x_-x

    if(np.linalg.norm(r)<tol):
      return np.diag(np.diag(x_))
    elif(np.linalg.norm(r)>np.linalg.norm(c)*1e5): 
      print("suspected overflow")
      return -1
    else:
      x=x_
      j+=1
  return "maxiter reached, no solution found"

In [61]:
A=np.random.standard_normal((5,5))

In [62]:
b=np.random.standard_normal(5)

In [63]:
x=jacobi(A,b,tol=np.linalg.norm(b)/100.)

suspected overflow


In [64]:
A_=A+np.diag(np.diag(A))*100. # the Jacobi iteration is designed for diagonally dominated matrices

In [65]:
x=jacobi(A_,b,tol=np.linalg.norm(b)/100.)

In [66]:
x

array([-0.01139502,  0.00365247,  0.00046505, -0.00326163,  0.01474043])

In [67]:
print("norm of b: %f\nnorm of error in Ax: %f\nnorm of true solution y: %f\nnorm of x-y:%f"
%(np.linalg.norm(b)
,np.linalg.norm(A_.dot(x)-b)
,np.linalg.norm(np.linalg.inv(A_).dot(b))
,np.linalg.norm(np.linalg.inv(A_).dot(b)-x)))

norm of b: 1.286767
norm of error in Ax: 0.001021
norm of true solution y: 0.019286
norm of x-y:0.000017


In [70]:
x=jacobi(A_,b,tol=np.linalg.norm(b)/10000.)

In [71]:
print("norm of b: %f\nnorm of error in Ax: %f\nnorm of true solution y: %f\nnorm of x-y:%f"
%(np.linalg.norm(b)
,np.linalg.norm(A_.dot(x)-b)
,np.linalg.norm(np.linalg.inv(A_).dot(b))
,np.linalg.norm(np.linalg.inv(A_).dot(b)-x)))

norm of b: 1.286767
norm of error in Ax: 0.000048
norm of true solution y: 0.019286
norm of x-y:0.000001


## 2) Gauss-Seidel iteration

Conceptually, the Gauss-Seidel method is very similar to the Jacobi. The difference is that instead of the diagonal's inverse, we use the inverse of the lower triangular part of the original $\mathbf{A}$ matrix.

The exact details of the algorithm were taken from example 7.9.

In [72]:

def gauss_seidel(A,b,tol,maxiter=10000):
  """
  perform Gauss-Seidel iteration on the system defined by A and b. Iterate until the L2 norm of the residual is smaller than >>tol<< but max >>maxiter<< times

  since we do not check wheter the method is convergent, I had to include manual criteria to prevent overflow: if the residual grows too much,
  the iteration is terminated, assuming it ain't convergent

  the notation resembles the one of example 7.9 in the lecture slides
  """
  
  for i in range(A.shape[0]):
    try:
      _=1./A[i,i]
    except:
      print('zero in diagonal, cannot apply Jacobi')
  
  

  k=0
  x=np.zeros(A.shape[0],float)

  while(k<maxiter):

    x_old=x.copy()
    
    # update of coordinates
    
    for i in range(x.shape[0]):
      x[i]=b[i]/A[i,i]

      for j in range(x.shape[0]):
        if(i==j):
          continue
        x[i]-=A[i,j]/A[i,i]*x[j] # due to our storage mechanism, this implementation is in fact the Gauss-Seidel
    
    r=x-x_old

    if(np.linalg.norm(r)<tol): # found solution
      return np.diag(np.diag(x))

    elif(np.linalg.norm(r)>np.linalg.norm(b)*1e5): # suspect overflow
      print("suspected overflow")
      return -1

    else: # next iteration
      k+=1

  return "maxiter reached, no solution found"





In [73]:
A=np.random.standard_normal((5,5))

In [74]:
b=np.random.standard_normal(5)

In [75]:
x=gauss_seidel(A,b,tol=np.linalg.norm(b)/100.)

suspected overflow


conjecture: the Gauss-Seidel works best for almost lower-triangular matrices

In [78]:
A_=A.copy()
for i in range(A.shape[0]):
  for j in range(i):
    A_[j,i]=A_[i,j]/1000.


In [79]:
x=gauss_seidel(A_,b,tol=np.linalg.norm(b)/100.)

In [80]:
print("norm of b: %f\nnorm of error in Ax: %f\nnorm of true solution y: %f\nnorm of x-y:%f"
%(np.linalg.norm(b)
,np.linalg.norm(A_.dot(x)-b)
,np.linalg.norm(np.linalg.inv(A_).dot(b))
,np.linalg.norm(np.linalg.inv(A_).dot(b)-x)))

norm of b: 3.596355
norm of error in Ax: 0.000032
norm of true solution y: 3129.866443
norm of x-y:0.005133


In [81]:
x=gauss_seidel(A_,b,tol=np.linalg.norm(b)/10000.)

In [82]:
print("norm of b: %f\nnorm of error in Ax: %f\nnorm of true solution y: %f\nnorm of x-y:%f"
%(np.linalg.norm(b)
,np.linalg.norm(A_.dot(x)-b)
,np.linalg.norm(np.linalg.inv(A_).dot(b))
,np.linalg.norm(np.linalg.inv(A_).dot(b)-x)))

norm of b: 3.596355
norm of error in Ax: 0.000000
norm of true solution y: 3129.866443
norm of x-y:0.000037


### 3) Newton method for scalar functions

The Newton method is a fairly simple algorithm for approximating the solution of the nonlinear scalar equation $f(x)=0$.

The update rule is: $x_{k+1}=x_{k}-f(x_k)/f'(x_k)$

It is easy to see that the method is based on the first order approximation of the function around the current $x$ point.

In general, the computation of the derivative is a nontrivial problem by itself. To solve this (well, rather to limit the amount of work that is needed) we will compute symbolic derivatives with the **theano** package. 

In [83]:
def newton_solver(f,df,x_0,tol,maxiter=10000):
  '''
  Newton  solver for f(x)=0, assuming that f,df are functions for f and its derivative

  stopping criterion: |f(x)|<tol
  '''

  x=x_0
  i=0

  while(i<maxiter):

    if(np.abs(f(x))<tol):
      return x

    try:
      x=x-f(x)/df(x)
    except:
      print("division by zero?")
      return -1

    i+=1
  
  print("maxiter reached, no soltion found")
  return -1




In [84]:
xx=tt.dscalar()
yy=xx**3-2*xx**2+10*np.sin(xx)**2
dyy=tt.grad(yy,xx)

f=theano.function([xx],yy)
df=theano.function([xx],dyy)

In [85]:
x_s=newton_solver(f,df,10.,1e-2)

In [86]:
f(x_s)

array(0.00560625)

In [87]:
x_s # 0 is actually solution

0.02643249272291941


### 4) Newton method for vector equations

The Newton method for vector equations is to solve the nonlinear vector equation 𝑓(𝑥)=0. In spirit, it is the straightorward generalizaion of the scalar method.

The update rule is: $x_{k+1}=x_k-(f'(x_k))^{-1}f(x_k)$

It is easy to see that the method is based on the first order approximation of the function around the current 𝑥 point as well.

For the Jacobian, we again rely on theano.


In [88]:
def newton_vector_solver(f,df,x_0,tol,maxiter=10000):
  '''
  Newton  solver for f(x)=0, assuming that f,df are functions for f and its Jacobian

  stopping criterion: |f(x)|<tol
  '''

  x=x_0
  i=0

  while(i<maxiter):

    if(np.linalg.norm(f(x))<tol):
      return x

    try:
      x=x-np.linalg.inv(df(x)).dot(f(x))
    except:
      print("singular Jacobian?")
      return -1
    i+=1
  
  print("maxiter reached, no soltion found")
  return -1



In [89]:
x=tt.dvector('x')
             
y=x*(1+x.sum()-x**2)
dy=theano.gradient.jacobian(y,x)

f=theano.function([x],y)
df=theano.function([x],dy)

In [90]:
x_s=newton_vector_solver(f,df,[10.,10.],1e-2)

In [91]:
x_s

array([2.41428556, 2.41428556])

In [92]:
np.linalg.norm(x_s-np.asarray([1+np.sqrt(2),1+np.sqrt(2)])) # the latter is actually a solution

0.0001018251269741721

In [93]:
np.linalg.norm(f(x_s)) 

0.0006953438961424595


# **Summary, discussion**

Based on the tests, we can conclude that all our methods work.

For the iterative methods, an extra layer of security had to be added to ensure that iterating with matrices that won't lead to convergence will not lead to overflow.

The Newton methods both worked well, despite the attempt to test on nontrivial functions. To compute symbolic derivatives, the **theano** is employed.